In [1]:
import nltk
import os
from nltk.corpus import stopwords
import string
import csv
from scipy.spatial.distance import cosine, cdist
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer 
from nltk.probability import FreqDist


path='D:\\College\\Semester 9\\NLP\\Lab\\FinalProject\\Tweets'
ps = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')
DocsTerms={}
stop_words=set(stopwords.words('english')) 
i=1
for root, dirs, files in os.walk(path):
    for file in files: # looping only over first 20,000 files
        
        with open(os.path.join(root,file), 'r', encoding="utf8") as f:
            reader = csv.reader(f)
            for row in reader:
                if not row[0] in DocsTerms :
                   # CheckifRetweet(row[10])
                    if " RT " not in row[10] or " rt " not in row[10] and  len(row[10]) > 20 : 
                        key =row[0] 
                        DocsTerms[key]=[]
                        DocsTerms[key].append(row[1])
                        LowerVocab=row[10].lower()
                        noLinkSent=re.sub(r'http\S+','',LowerVocab,flags=re.MULTILINE)
                        words = tokenizer.tokenize(noLinkSent)
                        stemmedVocab=[]
                        for word in words:
                            stemmedVocab.append(ps.stem(word))
                        filtered_sent=[w for w in stemmedVocab if not w in stop_words]
                        DocsTerms[key].append(filtered_sent)

DocsTerms.pop('tweet_id')


train={}
test={}
i=0
for key, value in DocsTerms.items():
    if i<=11712 :
        train[key]=[]
        train[key].append(value[0])
        train[key].append(value[1])
    else:
        test[key]=[]
        test[key].append(value[0])
        test[key].append(value[1])
    i=i+1


TfidfVectorizerFunc = TfidfVectorizer()
CountVectorizerFunc = CountVectorizer()
sentences = []
twitterId=[]
semitment=[]
for key, value in train.items():
    s=""
    twitterId.append(key)
    semitment.append(value[0])
    for word in  value[1]:
        s+=word+" "
    sentences.append(s)
    

tfidf = TfidfVectorizerFunc.fit_transform(sentences)

sentences_test = []
twitterId_test=[]
semitment_test=[]

for key, value in test.items():
    s=""
    twitterId_test.append(key)
    semitment_test.append(value[0])
    for word in  value[1]:
        s+=word+" "
    sentences_test.append(s)

#not needed
#TermDocIncidenceMatrix = CountVectorizerFunc.fit_transform(sentences)

#print(sentences)

#print(sorted(TfidfVectorizerFunc.vocabulary_.items(), key=lambda x : x[1]))
#for row in tfidf.toarray():
    #print(["%.4f"% val for val in row])

#print(sorted(CountVectorizerFunc.vocabulary_.items(), key=lambda x: x[1])) 
#print(TermDocIncidenceMatrix.toarray()[1])

#print(DocsTerms)

#print(train)

#print(test)

           

In [5]:
print(semitment)

['neutral', 'positive', 'neutral', 'negative', 'negative', 'negative', 'positive', 'neutral', 'positive', 'positive', 'neutral', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'negative', 'positive', 'positive', 'neutral', 'negative', 'negative', 'negative', 'neutral', 'negative', 'neutral', 'negative', 'neutral', 'negative', 'negative', 'positive', 'neutral', 'positive', 'positive', 'neutral', 'negative', 'positive', 'negative', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'negative', 'positive', 'positive', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'neutral', 'positive', 'neutral', 'negative', 'negative', 'positive', 'negative', 'neutral', 'neutral', 'neutral', 'negative', 'positive', 'positive', 'neutral', 'neutral', 'negative', 'neutral', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative',

In [2]:
from sklearn.naive_bayes import MultinomialNB
cats = ['neutral','positive','negative']
clf = MultinomialNB(alpha=1)
clf.fit(tfidf, semitment)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [3]:
import numpy as np
tfidf_test = TfidfVectorizerFunc.transform(sentences_test)
predictions = clf.predict(tfidf_test)


In [4]:
from sklearn import metrics
print(metrics.f1_score(semitment_test, predictions, average='micro'))

0.727994227994228


In [8]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(tfidf, semitment) 
predictions_neigh = neigh.predict(tfidf_test)
print(metrics.f1_score(semitment_test, predictions_neigh, average='micro'))

0.7297979797979797


In [7]:
from sklearn.ensemble import RandomForestClassifier
randomClassifier = RandomForestClassifier(random_state=0)
randomClassifier.fit(tfidf, semitment) 
predictions_randomClassifier = randomClassifier.predict(tfidf_test)
print(metrics.f1_score(semitment_test, predictions_randomClassifier, average='micro'))



0.7784992784992785


c:\users\yasmi\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [ ]:
#for row in tfidf.toarray():
#    print(FreqDist(row))

<FreqDist with 4 samples and 9767 outcomes>
<FreqDist with 7 samples and 9767 outcomes>
<FreqDist with 9 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 6 samples and 9767 outcomes>
<FreqDist with 16 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 3 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 15 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 3 samples and 9767 outcomes>
<FreqDist with 7 samples and 9767 outcomes>
<FreqDist with 16 samples and 9767 outcomes>
<FreqDist with 16 samples and 9767 outcomes>
<FreqDist with 3 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 5 samples and 9767 outcomes>
<FreqDist with 8 sampl

<FreqDist with 5 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 5 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 18 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 9 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 16 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 15 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 9 s

<FreqDist with 5 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 6 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 9 samples and 9767 outcomes>
<FreqDist with 6 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 4 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 15 samples and 9767 outcomes>
<FreqDist with 9 

<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 6 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 19 samples and 9767 outcomes>
<FreqDist with 3 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 7 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 9 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 9 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 9 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 6 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 13 s

<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 19 samples and 9767 outcomes>
<FreqDist with 15 samples and 9767 outcomes>
<FreqDist with 6 samples and 9767 outcomes>
<FreqDist with 15 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 15 samples and 9767 outcomes>
<FreqDist with 9 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 7 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 5 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 7 samples and 9767 outcomes>
<FreqDist with 

<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 9 samples and 9767 outcomes>
<FreqDist with 16 samples and 9767 outcomes>
<FreqDist with 4 samples and 9767 outcomes>
<FreqDist with 4 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 16 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with

<FreqDist with 15 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 9 samples and 9767 outcomes>
<FreqDist with 7 samples and 9767 outcomes>
<FreqDist with 5 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 17 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 7 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 7 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 5 

<FreqDist with 16 samples and 9767 outcomes>
<FreqDist with 15 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 7 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 18 samples and 9767 outcomes>
<FreqDist with 16 samples and 9767 outcomes>
<FreqDist with 6 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 15 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 6 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 16 samples and 9767 outcomes>
<FreqDist with 15 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 

<FreqDist with 9 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 6 samples and 9767 outcomes>
<FreqDist with 7 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 9 samples and 9767 outcomes>
<FreqDist with 17 samples and 9767 outcomes>
<FreqDist with 4 samples and 9767 outcomes>
<FreqDist with 9 samples and 9767 outcomes>
<FreqDist with 15 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 16 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 3 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 14 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 16 samples and 9767 outcomes>
<FreqDist with 14 s

<FreqDist with 16 samples and 9767 outcomes>
<FreqDist with 9 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 9 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 7 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 11 samples and 9767 outcomes>
<FreqDist with 4 samples and 9767 outcomes>
<FreqDist with 12 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 10 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 5 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 19 samples and 9767 outcomes>
<FreqDist with 8 samples and 9767 outcomes>
<FreqDist with 3 samples and 9767 outcomes>
<FreqDist with 13 samples and 9767 outcomes>
<FreqDist with 15 

In [ ]:
print(sentences)

In [3]:
import re
def CheckifRetweet(tweetText):
    regexp = re.compile(r'\b[R|r]t\b')
    if regexp.search(tweetText):
        print("yes")
    
    

In [5]:
print(DocsTerms)

{'570306133677760513': ['neutral', ['virginamerica', 'dhepburn', 'said']], '570301130888122368': ['positive', ['virginamerica', 'plu', 'ad', 'commerci', 'experi', 'tacki']], '570301083672813571': ['neutral', ['virginamerica', 'today', 'must', 'mean', 'need', 'take', 'anoth', 'trip']], '570301031407624196': ['negative', ['virginamerica', 'realli', 'aggress', 'blast', 'obnoxi', 'entertain', 'guest', 'face', 'amp', 'littl', 'recours']], '570300817074462722': ['negative', ['virginamerica', 'realli', 'big', 'bad', 'thing']], '570300767074181121': ['negative', ['virginamerica', 'serious', 'would', 'pay', '30', 'flight', 'seat', 'thi', 'play', 'realli', 'onli', 'bad', 'thing', 'fli', 'va']], '570300616901320704': ['positive', ['virginamerica', 'ye', 'nearli', 'everi', 'time', 'fli', 'vx', 'thi', 'ear', 'worm', 'go', 'away']], '570300248553349120': ['neutral', ['virginamerica', 'realli', 'miss', 'prime', 'opportun', 'men', 'without', 'hat', 'parodi']], '570299953286942721': ['positive', ['virg